##🎯 What You'll Learn:

✅ Upload and manage shapefiles (study area and Egypt boundaries) as Earth Engine assets

✅ Load shapefiles into Colab using ee.FeatureCollection()

✅ Filter and visualize Landsat 9 images by study area or selected governorates



In [ ]:
# Import Libraries
import ee
import geemap
import pandas as pd

In [ ]:
# Authenticate to Earth Engine.
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project =  'your_project_name'  )

Load FeatureCollections from Assets

In [ ]:
# Load your study area shapefile
study_area = ee.FeatureCollection("users/your_username/study_area")

In [ ]:
# Create an interactive map
Map = geemap.Map(width = 800)
Map.centerObject(study_area, 12)

# Add layers
Map.add_layer(study_area, {}, "Study Area")
Map

Get Landsat 9 Image for the Study Area

In [ ]:
# Load and filter Landsat 9 imagery by study area
# For best single image
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(study_area)
    .filterDate("2024-01-01", "2025-01-01")
    .filter(ee.Filter.lt("CLOUD_COVER", 10))
    .sort("CLOUD_COVER")
    .sort("system:time_start", False)      # Sort by date in descending order
    .first()
)

In [ ]:
# Define visualization parameters
true_color = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],   # Red, Green, Blue
    'min': 5000,
    'max': 15000,
}

false_color = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],   # NIR, Red, Green
    'min': 5000,
    'max': 15000,
}

In [ ]:
# Add the layers to the map
Map.add_layer(image,true_color, "Study Area True Color")
Map.add_layer(image,false_color, "Study Area False Color")

Map

Load Egypt Admin-1 Shapefile from Assets

In [ ]:
# Load Egypt governorates shapefile
egypt_admin = ee.FeatureCollection("users/your_username/egypt_admin1")

In [ ]:
# Add Egypt governorates to the same map
Map.centerObject(egypt_admin, 6)
Map.add_layer(egypt_admin, {}, "Egypt Governorates")
Map

View the properties of your FeatureCollection

In [ ]:
# Print first feature as dictionary
first_feature = egypt_admin.first()
print(first_feature.getInfo())

In [ ]:
# Get a list of all values of shapeName
names = egypt_admin.aggregate_array("shapeName").getInfo()
print(names)

In [ ]:
# Convert FeatureCollection to DataFrame
df = geemap.ee_to_df(egypt_admin)

df.head()

In [ ]:
# List all unique governorate names
print(df['shapeName'].unique())

In [ ]:
gov_names = [ 'Alexandria Governorate' , 'Beheira Governorate', 'Minya Governate' ]
selected_gov = egypt_admin.filter(ee.Filter.inList("shapeName", gov_names))
Map.centerObject(selected_gov, 6)
Map.addLayer(selected_gov, {}, "Selected Governorates")
Map

Get Landsat 9 Image for the selcted governorates

In [ ]:
 # Filter Landsat imagery by selected governorate
image_sel_gov = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(selected_gov)
    .filterDate("2024-01-01", "2025-01-01")
    .filter(ee.Filter.lt("CLOUD_COVER", 10))
    .median()
)

In [ ]:
# Add the layers to the map
Map.add_layer(image_sel_gov,true_color, "Sel_Govs True Color")
Map.add_layer(image_sel_gov,false_color, "Sel_Govs False Color")

Map